In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
movies = pd.read_csv('/Users/gost1/Desktop/Recommender systems/Downloads/movies.csv')
ratings = pd.read_csv('/Users/gost1/Desktop/Recommender systems/Downloads/ratings.csv')
links = pd.read_csv('/Users/gost1/Desktop/Recommender systems/Downloads/links.csv')
tags = pd.read_csv('/Users/gost1/Desktop/Recommender systems/Downloads/tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [6]:
df = movies.merge(ratings.assign(Вratings=ratings['movieId'].astype('float32')), on=['movieId'], how='left')

In [7]:
mean_movie = df[['movieId', 'rating']].groupby(['movieId']).mean()

In [8]:
df = df.merge(mean_movie, on='movieId', how='left')

In [9]:
df = df.rename(columns={df.columns[7]: "mean_movie" })

In [10]:
mean_user = ratings[['userId', 'rating']].groupby(['userId']).mean()

In [11]:
df = df.merge(mean_user, on='userId', how='left')

In [13]:
df = df.rename(columns={df.columns[8]: "mean_user" })

In [14]:
df.head()

,movieId,title,genres,userId,rating_x,timestamp,Вratings,mean_movie,mean_user
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,1.0,3.92093,4.366379
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,1.0,3.92093,3.636364
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,1.0,3.92093,3.230263
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,1.0,3.92093,3.448148
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,1.0,3.92093,4.209524


In [15]:
df.drop(['title', 'genres', 'Вratings'], axis=1, inplace=True)

In [16]:
df.dropna(inplace=True)

In [17]:
Y = df['rating_x']

In [18]:
df.drop(['rating_x'], axis=1, inplace=True)
X = df

In [19]:
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)

In [20]:
model.fit(X_train, y_train)

LinearRegression()

In [21]:
y_pred = model.predict(X_test)

In [22]:
RMSE = mean_squared_error(y_test, y_pred, squared=False)

In [23]:
print("RMSE =", RMSE)

RMSE = 0.811190651185146
